In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('data/trainset.csv', index_col=0)
    test  = pd.read_csv('data/testset.csv', index_col=0)
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).sum()
        test[col+'w'] = test[col].rolling(window=window).sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier 

In [4]:
def train_classifier(X, Xt, y, yt):
    
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  LogisticRegression(),
                  CatBoostClassifier()]
    params1 = {  
        "n_estimators": st.randint(10,500),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 50),     # Maximum tree depth for base learners.
    }
    
    params2 = {  
        "n_estimators": st.randint(10,500),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 50),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    params4 = {}
    
    model_params = [params1, params2, params2, params3, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=kf,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results.sort_values(by='Test Score', ascending=False)

In [10]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 211 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   23.4s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    1.4s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

160:	learn: 0.6274671	total: 1.19s	remaining: 6.21s
161:	learn: 0.6270492	total: 1.2s	remaining: 6.2s
162:	learn: 0.6267586	total: 1.21s	remaining: 6.22s
163:	learn: 0.6265640	total: 1.22s	remaining: 6.22s
164:	learn: 0.6261956	total: 1.23s	remaining: 6.21s
165:	learn: 0.6260879	total: 1.23s	remaining: 6.19s
166:	learn: 0.6258557	total: 1.24s	remaining: 6.17s
167:	learn: 0.6255531	total: 1.24s	remaining: 6.16s
168:	learn: 0.6251398	total: 1.25s	remaining: 6.17s
169:	learn: 0.6247947	total: 1.26s	remaining: 6.18s
170:	learn: 0.6245675	total: 1.27s	remaining: 6.16s
171:	learn: 0.6243408	total: 1.28s	remaining: 6.15s
172:	learn: 0.6237115	total: 1.28s	remaining: 6.13s
173:	learn: 0.6234237	total: 1.29s	remaining: 6.11s
174:	learn: 0.6231644	total: 1.29s	remaining: 6.1s
175:	learn: 0.6226944	total: 1.3s	remaining: 6.11s
176:	learn: 0.6224098	total: 1.32s	remaining: 6.12s
177:	learn: 0.6221540	total: 1.32s	remaining: 6.11s
178:	learn: 0.6219925	total: 1.33s	remaining: 6.09s
179:	learn: 0.62

319:	learn: 0.5939687	total: 2.17s	remaining: 4.62s
320:	learn: 0.5938857	total: 2.18s	remaining: 4.61s
321:	learn: 0.5937596	total: 2.18s	remaining: 4.6s
322:	learn: 0.5936403	total: 2.19s	remaining: 4.59s
323:	learn: 0.5933327	total: 2.19s	remaining: 4.58s
324:	learn: 0.5931396	total: 2.2s	remaining: 4.57s
325:	learn: 0.5931058	total: 2.21s	remaining: 4.56s
326:	learn: 0.5929600	total: 2.21s	remaining: 4.55s
327:	learn: 0.5928030	total: 2.22s	remaining: 4.54s
328:	learn: 0.5926359	total: 2.22s	remaining: 4.54s
329:	learn: 0.5922488	total: 2.23s	remaining: 4.53s
330:	learn: 0.5917985	total: 2.24s	remaining: 4.52s
331:	learn: 0.5915419	total: 2.24s	remaining: 4.51s
332:	learn: 0.5914011	total: 2.25s	remaining: 4.5s
333:	learn: 0.5913407	total: 2.25s	remaining: 4.49s
334:	learn: 0.5911519	total: 2.26s	remaining: 4.48s
335:	learn: 0.5908733	total: 2.26s	remaining: 4.47s
336:	learn: 0.5907541	total: 2.27s	remaining: 4.47s
337:	learn: 0.5907383	total: 2.28s	remaining: 4.46s
338:	learn: 0.5

495:	learn: 0.5711731	total: 3.35s	remaining: 3.41s
496:	learn: 0.5711400	total: 3.36s	remaining: 3.4s
497:	learn: 0.5709923	total: 3.36s	remaining: 3.39s
498:	learn: 0.5707504	total: 3.37s	remaining: 3.38s
499:	learn: 0.5705988	total: 3.38s	remaining: 3.38s
500:	learn: 0.5704614	total: 3.38s	remaining: 3.37s
501:	learn: 0.5704338	total: 3.4s	remaining: 3.37s
502:	learn: 0.5703666	total: 3.41s	remaining: 3.37s
503:	learn: 0.5702390	total: 3.42s	remaining: 3.37s
504:	learn: 0.5701102	total: 3.44s	remaining: 3.37s
505:	learn: 0.5700334	total: 3.45s	remaining: 3.37s
506:	learn: 0.5698358	total: 3.46s	remaining: 3.37s
507:	learn: 0.5697087	total: 3.47s	remaining: 3.36s
508:	learn: 0.5694877	total: 3.47s	remaining: 3.35s
509:	learn: 0.5692148	total: 3.48s	remaining: 3.34s
510:	learn: 0.5691211	total: 3.48s	remaining: 3.33s
511:	learn: 0.5690103	total: 3.49s	remaining: 3.33s
512:	learn: 0.5690017	total: 3.5s	remaining: 3.32s
513:	learn: 0.5689525	total: 3.5s	remaining: 3.31s
514:	learn: 0.56

685:	learn: 0.5494781	total: 4.53s	remaining: 2.07s
686:	learn: 0.5494163	total: 4.54s	remaining: 2.07s
687:	learn: 0.5493234	total: 4.54s	remaining: 2.06s
688:	learn: 0.5491322	total: 4.55s	remaining: 2.05s
689:	learn: 0.5489872	total: 4.55s	remaining: 2.05s
690:	learn: 0.5489139	total: 4.56s	remaining: 2.04s
691:	learn: 0.5488434	total: 4.57s	remaining: 2.03s
692:	learn: 0.5487848	total: 4.57s	remaining: 2.02s
693:	learn: 0.5487785	total: 4.58s	remaining: 2.02s
694:	learn: 0.5486990	total: 4.58s	remaining: 2.01s
695:	learn: 0.5485725	total: 4.59s	remaining: 2s
696:	learn: 0.5485394	total: 4.59s	remaining: 2s
697:	learn: 0.5484714	total: 4.6s	remaining: 1.99s
698:	learn: 0.5483728	total: 4.6s	remaining: 1.98s
699:	learn: 0.5482942	total: 4.61s	remaining: 1.98s
700:	learn: 0.5482219	total: 4.62s	remaining: 1.97s
701:	learn: 0.5481684	total: 4.62s	remaining: 1.96s
702:	learn: 0.5481270	total: 4.63s	remaining: 1.95s
703:	learn: 0.5480373	total: 4.63s	remaining: 1.95s
704:	learn: 0.547951

856:	learn: 0.5308861	total: 5.5s	remaining: 917ms
857:	learn: 0.5308366	total: 5.5s	remaining: 911ms
858:	learn: 0.5307974	total: 5.51s	remaining: 904ms
859:	learn: 0.5307247	total: 5.51s	remaining: 898ms
860:	learn: 0.5306526	total: 5.52s	remaining: 891ms
861:	learn: 0.5304585	total: 5.53s	remaining: 885ms
862:	learn: 0.5304234	total: 5.53s	remaining: 878ms
863:	learn: 0.5303821	total: 5.54s	remaining: 872ms
864:	learn: 0.5303518	total: 5.54s	remaining: 865ms
865:	learn: 0.5303504	total: 5.55s	remaining: 858ms
866:	learn: 0.5302647	total: 5.55s	remaining: 852ms
867:	learn: 0.5302644	total: 5.56s	remaining: 845ms
868:	learn: 0.5301554	total: 5.56s	remaining: 839ms
869:	learn: 0.5300962	total: 5.57s	remaining: 832ms
870:	learn: 0.5300784	total: 5.58s	remaining: 826ms
871:	learn: 0.5300080	total: 5.58s	remaining: 819ms
872:	learn: 0.5299574	total: 5.59s	remaining: 813ms
873:	learn: 0.5298909	total: 5.59s	remaining: 806ms
874:	learn: 0.5297911	total: 5.6s	remaining: 800ms
875:	learn: 0.5

In [16]:
resultado.sort_values(by='Train Score', ascending=False).iloc[:, :-1]

In [17]:
resultado

,Model,Window,Neutral,Train Score
0,Random Forest,3,1,0.640200
4,Catboost,3,1,0.625289
0,Random Forest,3,0,0.617733
2,LightGBM,3,1,0.616964
1,XGBoost,3,1,0.615322
0,Random Forest,4,1,0.613147
1,XGBoost,4,1,0.602068
3,Logistic Regression,1,0,0.596374
2,LightGBM,4,1,0.595002
2,LightGBM,1,0,0.594916
